# Data Science Case Study - Batch 9


### OBJECTIVE - This case study is around predicting whether a patient is suffering from Diabetes based on certain Diagnostic Measurements.

### Data Loading and Cleansing 

#### Step 1 - Import the libraries that will be used in the notebook

In [1]:
import numpy as np
import pandas as pd

#### Step 2 - Read the CSV file and see the first five rows of data

In [2]:
df = pd.read_csv("Diabetes.csv",header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### Step 3 - As we can see that data is not having the header so we will specify the header to the data frame.

 0. Number of times pregnant
 1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
 2. Diastolic blood pressure (mm Hg)
 3. Triceps skin fold thickness (mm)
 4. 2-Hour serum insulin (mu U/ml)
 5. Body mass index (weight in kg/(height in m)^2)
 6. Diabetes pedigree function
 7. Age (years)
 8. Class variable (0 or 1)


In [3]:
df.columns = ["NoTimePregnant", "GlucoseConcentration", "DiastolicBloodPressure", "TricepsSkinThickness","2HourSerumInsulin","BMI"
             ,"DPF","Age","Class"]
df.head()

,NoTimePregnant,GlucoseConcentration,DiastolicBloodPressure,TricepsSkinThickness,2HourSerumInsulin,BMI,DPF,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### Step 4 - Lets see what all cleansing is required

In [4]:
df.describe()

,NoTimePregnant,GlucoseConcentration,DiastolicBloodPressure,TricepsSkinThickness,2HourSerumInsulin,BMI,DPF,Age,Class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


#### We see that min GlucoseConcentration and Diastolic BloodPressure is 0 which is not correct data. So we have to identify such cases and fix them

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
NoTimePregnant            768 non-null int64
GlucoseConcentration      768 non-null int64
DiastolicBloodPressure    768 non-null int64
TricepsSkinThickness      768 non-null int64
2HourSerumInsulin         768 non-null int64
BMI                       768 non-null float64
DPF                       768 non-null float64
Age                       768 non-null int64
Class                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


#### Check if there is any NaN value.

In [12]:
df.isnull().values.any()

False

#### Check fo duplicated row.

In [13]:
sum(df.duplicated())

0

In [27]:
df.loc[~((df.GlucoseConcentration == 0) | (df.TricepsSkinThickness == 0) | (df.BMI == 0) | (df.DiastolicBloodPressure ==0))].describe()

,NoTimePregnant,GlucoseConcentration,DiastolicBloodPressure,TricepsSkinThickness,2HourSerumInsulin,BMI,DPF,Age,Class
count,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000,532.000000
mean,3.516917,121.030075,71.505639,29.182331,114.988722,32.890226,0.502966,31.614662,0.332707
std,3.312036,30.999226,12.310253,10.523878,123.007555,6.881109,0.344546,10.761584,0.471626
min,0.000000,56.000000,24.000000,7.000000,0.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,98.750000,64.000000,22.000000,0.000000,27.875000,0.258750,23.000000,0.000000
50%,2.000000,115.000000,72.000000,29.000000,91.500000,32.800000,0.416000,28.000000,0.000000
75%,5.000000,141.250000,80.000000,36.000000,165.250000,36.900000,0.658500,38.000000,1.000000
max,17.000000,199.000000,110.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000
